# Assignment

In this assignment we will train a convolutional neural network (CNN) on the CIFAR-10 dataset.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook (push to https://github.com/[username]/cs190/cnn/assignment.ipynb)
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [46]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [0]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [0]:
# --- Select Tensorflow 2.0 (only in Google Colab)
% tensorflow_version 2.x

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [15]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

### Imports

Use the following lines to import any additional needed libraries:

In [0]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from jarvis.train import datasets

# Data

As in the tutorial, data for this assignment will consist of the CIFAR-10 dataset comprising 10 different everyday objects (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck). The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset
3. Prepare the corresponding Tensorflow Input(...) objects for model definition

In [17]:
# --- Download dataset
datasets.download(name='cifar')

# --- Prepare generators and model inputs
gen_train, gen_valid, client = datasets.prepare(name='cifar')
inputs = client.get_inputs(Input)

[ 2020-04-20 07:47:06 ] [====================] 100.000% : Iterating | 000001    

# Training

In this assignment we will train a basic convolutional neural network on the CIFAR-10 dataset. At minumum you must include the following baseline techniques covered in the tutorial:

* convolutional operations
* batch normalization
* activation function
* subsampling

You are also **encouraged** to try different permuations and customizations to achieve optimal validation accuracy.

### Define the model

In [38]:
from tensorflow.keras import regularizers

kwargs = {
    'kernel_size': (3, 3),
    'padding': 'same'
    'kernel_regularizer': regularizers.l2(0.01)}

relu = lambda x: layers.LeakyReLU()(x)
norm = lambda x: layers.BatchNormalization()(x)
conv = lambda x, filters, strides : layers.Conv2D(filters=filters, strides=strides, **kwargs)(x)
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(2, 2))))


SyntaxError: ignored

In [0]:
# --- Define dropout
drop = layers.Dropout(rate=0.25)

# --- Define series of blocks
l1 = conv1(16, inputs['dat'])
l2 = conv1(24, conv2(24, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
#print (l4.shape)

f0 = layers.Flatten()(l4)
#print (f0.shape)
hidden_size = 64
f1 = drop(layers.Dense(hidden_size, activation='relu')(f0))

### Compile the model

In [0]:
# --- Compile model
logits = {}
logits['class'] = layers.Dense(10, name='class')(f1)

model = Model(inputs=inputs, outputs=logits)

# --- Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4), 
    loss={'class': losses.SparseCategoricalCrossentropy(from_logits=True)}, 
    metrics={'class': 'sparse_categorical_accuracy'})

### Train the model

In [43]:
model.fit(
    x=gen_train, 
    steps_per_epoch=250, 
    epochs=20,
    validation_data=gen_valid,
    validation_steps=250,
    validation_freq=4)

Epoch 1/20
250/250 [==============================] - 7s 27ms/step - loss: 2.1229 - sparse_categorical_accuracy: 0.2306
Epoch 2/20
250/250 [==============================] - 7s 27ms/step - loss: 1.8391 - sparse_categorical_accuracy: 0.3406
Epoch 3/20
250/250 [==============================] - 7s 28ms/step - loss: 1.7013 - sparse_categorical_accuracy: 0.3868
Epoch 4/20
250/250 [==============================] - 13s 53ms/step - loss: 1.6314 - sparse_categorical_accuracy: 0.4146 - val_loss: 1.5239 - val_sparse_categorical_accuracy: 0.4493
Epoch 5/20
250/250 [==============================] - 7s 27ms/step - loss: 1.5487 - sparse_categorical_accuracy: 0.4424
Epoch 6/20
250/250 [==============================] - 7s 28ms/step - loss: 1.4873 - sparse_categorical_accuracy: 0.4629
Epoch 7/20
250/250 [==============================] - 7s 28ms/step - loss: 1.4233 - sparse_categorical_accuracy: 0.4911
Epoch 8/20
250/250 [==============================] - 13s 52ms/step - loss: 1.4064 - sparse_catego

# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `model.predict(...)` on the data aggregated via a test generator.

In [44]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True)

# --- Aggregate all examples
xs = []
ys = []

for x, y in test_valid:
    xs.append(x['dat'])
    ys.append(y['class'])

xs = np.concatenate(xs)
ys = np.concatenate(ys)
# --- Predict
logits = model.predict(xs)

if type(logits) is dict:
    logits = logits['class']

# --- Argmax
pred = np.argmax(logits, axis=1)

[ 2020-04-20 08:37:58 ] [====================] 100.000% : Iterating | 012000    

**Note**: this cell is used only to check for model performance. It will not be graded. Once you are satisfied with your model, proceed to submission of your assignment below.

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort statistics. There is no need to submit training performance accuracy. As in the tutorial, ensure that there are at least three columns in the `*.csv` file:

* true (ground-truth)
* pred (prediction)
* corr (correction prediction, True or False)

In [0]:
# --- Create *.csv
df = pd.DataFrame(index=np.arange(pred.size))

# --- Define columns
df['true'] = ys[:, 0]
df['pred'] = pred
df['corr'] = df['true'] == df['pred']

In [0]:
# --- Serialize *.csv
fname = '{}/models/cnn/results.csv'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
df.to_csv(fname)

# Submission

Use the following line to save your model for submission (in Google Colab this should save your model file into your personal Google Drive):

In [0]:
# --- Serialize a model
fname = '{}/models/cnn/final.hdf5'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
model.save(fname)

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.